In [1]:
import mne as mne
import numpy as np
import seaborn as sns
from scipy import stats
import pandas as pd
import os as os
import nolds as nolds

In [2]:
def hfd(a):
    k_max=5
    L = []
    x = []
    N = a.size

    for k in range(1,k_max):
        Lk = 0
        for m in range(0,k):
            #we pregenerate all idxs
            idxs = np.arange(1,int(np.floor((N-m)/k)),dtype=np.int32)

            Lmk = np.sum(np.abs(a[m+idxs*k] - a[m+k*(idxs-1)]))
            Lmk = (Lmk*(N - 1)/(((N - m)/ k)* k)) / k
            Lk += Lmk


        L.append(np.log(Lk/(m+1)))
        x.append([np.log(1.0/ k), 1])

    p, r1, r2, s=np.linalg.lstsq(x, L)
    return p[0]

In [3]:
def hjorth(a):

    first_deriv = np.diff(a)
    second_deriv = np.diff(a,2)

    var_zero = np.mean(a ** 2)
    var_d1 = np.mean(first_deriv ** 2)
    var_d2 = np.mean(second_deriv ** 2)

    activity = var_zero
    morbidity = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / morbidity

    return activity, morbidity, complexity

In [4]:
def pfd(a):

    diff = np.diff(a)
    # x[i] * x[i-1] for i in t0 -> tmax
    prod = diff[1:-1] * diff[0:-2]

    # Number of sign changes in derivative of the signal
    N_delta = np.sum(prod < 0)
    n = len(a)

    return np.log(n)/(np.log(n)+np.log(n/(n+0.4*N_delta)))

In [5]:
def do_univariate(rawdata):
    pctiles=rawdata.apply(stats.zscore).describe(percentiles=
                                                 np.concatenate([[0.001,0.01],np.arange(.1,1,.1),[0.99, 0.999]]))
    pctiles.loc['skew',:]=rawdata.apply(stats.zscore).skew().values
    pctiles.loc['kurtosis',:]=rawdata.apply(stats.zscore).kurtosis().values
    pctiles.loc['hyperskew',:]=rawdata.apply(stats.zscore).apply(stats.moment,moment=5).values
    pctiles.loc['nan%',:]=np.sum(rawdata.isnull().any()) /rawdata.shape[1]
    
    pctiles.loc['hurst',:]=rawdata.apply(nolds.hurst_rs)
    pctiles.loc['pfd',:]=rawdata.apply(stats.zscore).apply(pfd)
    #pctiles.loc['dfa',:]=rawdata.apply(nolds.dfa)
    
    pctiles.loc['hj2',:]=rawdata.apply(hjorth).iloc[1,:]
    pctiles.loc['hj3',:]=rawdata.apply(hjorth).iloc[2,:]

    pctiles.drop(['mean','min','max','std','count'],inplace=True)
    pctiles=pd.melt(pctiles.reset_index(), id_vars='index')
    pctiles['name'] = pctiles[['variable','index']].apply(lambda x: '_'.join(x), axis=1)
    pctiles.drop(columns=['variable','index'],inplace=True)
    pctiles=pctiles.set_index('name').T
    pctiles.rename(index={'value': thisfile},inplace=True)
    pctiles = pctiles.rename_axis(None, axis = 1)
    return pctiles

In [6]:
def do_base_bivariate(epochs):
    con_all, freqs, _, _, _ =mne.connectivity.spectral_connectivity(epochs,sfreq=info['sfreq'],verbose=False)
    return con_all, freqs

In [7]:
def do_maxmin_bivariate(epochs):
    tensor=list()
    for idx in np.arange(0,len(epochs.events)):
        temp, freqs, _, _, _ =mne.connectivity.spectral_connectivity(epochs[idx],sfreq=info['sfreq'],verbose=False)
        tensor.append(temp)
    con_max= np.stack(tensor, axis=3).max(axis=3)
    con_min= np.stack(tensor, axis=3).min(axis=3)
    return con_max, con_min

In [8]:
def do_fft(raw):
    normfft,f=mne.time_frequency.psd_multitaper(raw,normalization='full',verbose=False)
    return normfft,f

In [9]:
df_all=pd.DataFrame()

filespace=os.listdir('/media/jadhavlab/Data2/Volcanoes/predict-volcanic-eruptions-ingv-oe/test/')
for file in filespace:
    thisfile=file.split('.csv')[0]
    rawdata= pd.read_csv('/media/jadhavlab/Data2/Volcanoes/predict-volcanic-eruptions-ingv-oe/test/'+file)
    
    # clean bad columns
    rawdata.loc[:,(rawdata == 0).all(axis=0)]=np.nan
    
    mask = np.isnan(rawdata)
    masked_df = np.ma.masked_array(rawdata, mask)
    fill_value = pd.DataFrame({col: rawdata.mean(axis=1) for col in rawdata.columns})
    rawdata2 = masked_df.filled(fill_value)
    rawdata = pd.DataFrame(data=rawdata2, columns=rawdata.columns)

    # get univariate data
    pctiles=do_univariate(rawdata)
    
    # zscore
    data= stats.zscore(rawdata.values)
    # idxs of one matrix page
    relidx=np.tril_indices(data.shape[1],-1)
    
    # made mne array
    info = mne.create_info(ch_names=[str(x+1) for x in range(data.shape[1])],
                       sfreq=200., ch_types=['eeg']*data.shape[1],verbose=False)
    raw = mne.io.RawArray(data.T, info,verbose=False)
    events = mne.make_fixed_length_events(raw, duration=1.)
    epochs = mne.Epochs(raw, events, tmin=0.0, tmax=1.0,baseline=None,verbose=False)
    
    # do fft
    normfft,f=do_fft(raw);
    # do coherence of all pairs
    con_all, freqs=do_base_bivariate(epochs)
    # do min/max coherence
    con_max, con_min=do_maxmin_bivariate(epochs)
    
    sizeofmat=(relidx[0].shape[0],con_all.shape[2])
    
    reducedcon_all=np.zeros(sizeofmat)
    reducedcon_min=np.zeros(sizeofmat)
    reducedcon_max=np.zeros(sizeofmat)
    
    for i in range(con_all.shape[2]):
        temp_all= con_all[:,:,i]
        temp_min= con_min[:,:,i]
        temp_max= con_max[:,:,i]

        reducedcon_all[:,i]=temp_all[relidx]
        reducedcon_min[:,i]=temp_min[relidx]
        reducedcon_max[:,i]=temp_max[relidx]
        
    basepair=list(map('-'.join,zip(list(map(str,relidx[0]+1)),list(map(str,relidx[1]+1)))))
    basefreq=list(map(str,np.round(freqs).astype(int)))
    replbasepair=basepair*sizeofmat[1]
    replbasefreq=list(np.repeat(basefreq,sizeofmat[0]))
    
    columnnames_all=list(map('-all-'.join,zip(replbasepair,replbasefreq)))
    columnnames_min=list(map('-min-'.join,zip(replbasepair,replbasefreq)))
    columnnames_max=list(map('-max-'.join,zip(replbasepair,replbasefreq)))
    columnnames_fft=list(map(str,f.astype(float)))
    
    columnnames=columnnames_all+columnnames_min+columnnames_max+columnnames_fft
    
    df = pd.DataFrame(columns=columnnames)
    df.loc[0]=np.concatenate((reducedcon_all.flatten(order='F'),
               reducedcon_min.flatten(order='F'),
               reducedcon_max.flatten(order='F'),
               normfft.mean(axis=0)))
    df.rename(index={0: thisfile},inplace=True)
    df=df.join(pctiles)
    df_all=df_all.append(df)
    print(thisfile +' done')

278061079 done
1156478311 done
724991886 done
540365975 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/mne/connectivity/spectral.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.con_scores[con_idx] = np.abs(csd_mean) / np.sqrt(psd_xx * psd_yy)


143490047 done
1507486833 done
1832091009 done
1139538400 done
1478980435 done
762441093 done
888261646 done
1275413550 done
704203472 done
1464708932 done
1476140509 done
2134667434 done
1261276830 done
516982827 done
874843071 done
795714457 done
525293176 done
34157657 done
929460995 done
301666308 done
1747934768 done
714397715 done
1202937149 done
1680042910 done
2050966683 done
1699141889 done
545287552 done
232867375 done
497525599 done
1324381186 done
2036676417 done
1277130036 done
898958179 done
1508620001 done
1811799403 done
852999346 done
941679734 done
331454611 done
825599858 done
446460497 done
1091745945 done
2112746396 done
1514310552 done
1660661980 done
1542688402 done
762138217 done
724696480 done
1249055479 done
1202189440 done
1756765333 done
33458659 done
786584481 done
2028882784 done
1414827062 done
573510041 done
1147881605 done
1007233480 done
1415551121 done
459658475 done
424084225 done
2136902826 done
872010061 done
613508749 done
789491002 done
839379496

/home/jadhavlab/anaconda3/lib/python3.7/site-packages/mne/connectivity/spectral.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.con_scores[con_idx] = np.abs(csd_mean) / np.sqrt(psd_xx * psd_yy)


1566091785 done
1798763550 done
1574093529 done
698018079 done


...

1441883863 done
1543187222 done
147575332 done
1356168880 done
1458603236 done
1918223103 done
617652607 done
1736854089 done
63625893 done
1161198334 done
375866434 done
51014357 done
298096929 done
1152668140 done
2105636637 done
1968971812 done
79788076 done
1064227773 done
1593670014 done
1741209207 done
764592471 done
1545502551 done
909234048 done
1193187793 done
276991251 done
850397684 done
1373993427 done
615355771 done
237189175 done
1413109088 done
1906018481 done
1438500383 done
855797438 done
530592602 done
797705280 done
2052946195 done
91840882 done
1663794609 done
346081225 done
220893709 done
429014193 done
81077420 done
489905928 done
1570294420 done
1611086980 done
366913729 done
349276058 done
1381951281 done
2083322756 done
126243235 done
1003545497 done
647855417 done
1058988926 done
951055471 done
1126221533 done
1195836508 done
1349669233 done
2030007913 done
1404653895 done
286415922 done
1736527312 done
1442677643 done
429876190 done
430788826 done
1974334562 

/home/jadhavlab/anaconda3/lib/python3.7/site-packages/mne/connectivity/spectral.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.con_scores[con_idx] = np.abs(csd_mean) / np.sqrt(psd_xx * psd_yy)


1454666803 done
1038229705 done
889325006 done
110836519 done
1755887993 done
2021536071 done
116714199 done
1205814686 done
1675650279 done
4678581 done
974071693 done
706971403 done
711484554 done
1316204196 done
575115873 done
2120732368 done
692818572 done
1231863231 done
1532453604 done
766739031 done
1950701043 done
1208675647 done
36637286 done
1964692006 done
129209722 done
83637460 done
1060953646 done
1999102785 done
211924631 done
1289291739 done
1925624997 done
75963900 done
1561964580 done
1670185458 done
1551124200 done
432656055 done
732966999 done
14464594 done


/home/jadhavlab/anaconda3/lib/python3.7/site-packages/mne/connectivity/spectral.py:91: RuntimeWarning: invalid value encountered in true_divide
  self.con_scores[con_idx] = np.abs(csd_mean) / np.sqrt(psd_xx * psd_yy)


1854574855 done
1212393638 done
1084581069 done
180533806 done
971806068 done
578822373 done
922141654 done
638139304 done
315554490 done
242192487 done
1122574109 done
1807322886 done
1072278088 done
1336360883 done
1751562628 done
1198404856 done
11785683 done
1482745022 done
2076290808 done
869834621 done
1811447173 done
2138795995 done
1112725405 done
1901468420 done
998136924 done
803075067 done
1006540019 done
1460070291 done
902806546 done
829021178 done
1190219966 done
832911243 done
1715687089 done
80878937 done
955142815 done
318604031 done
1661154530 done
1236374724 done
1255059548 done
383644444 done
3007537 done
515532915 done
391645929 done
1866698002 done
1003310246 done
1639452758 done
114150551 done
620920171 done
921110033 done
567868974 done
772722335 done


In [10]:
df_all

,2-1-all-6,3-1-all-6,3-2-all-6,4-1-all-6,4-2-all-6,4-3-all-6,5-1-all-6,5-2-all-6,5-3-all-6,5-4-all-6,...,sensor_10_99%,sensor_10_99.9%,sensor_10_skew,sensor_10_kurtosis,sensor_10_hyperskew,sensor_10_nan%,sensor_10_hurst,sensor_10_pfd,sensor_10_hj2,sensor_10_hj3
278061079,0.037354,0.067475,0.020187,0.031299,0.014989,0.124805,0.265643,0.668812,0.257345,0.266497,...,2.351133,3.249279,-0.049722,0.267869,-0.689045,0.0,0.333069,1.003794,0.156701,2.235447
1156478311,0.018182,0.036769,0.024426,0.098822,0.019722,0.138947,0.274468,0.703359,0.199588,0.243230,...,2.332292,3.251500,-0.018550,0.156780,0.089290,0.0,0.427414,1.004995,0.160326,3.434792
724991886,0.017491,0.031150,0.041356,0.046457,0.010241,0.052260,0.302501,0.549414,0.261949,0.347997,...,2.462663,3.688366,-0.032228,2.503571,5.035874,0.0,1.016827,1.015407,0.410927,3.112056
540365975,0.323143,0.323143,1.000000,0.018145,0.373922,0.373922,0.018812,0.209577,0.209577,0.026951,...,2.390581,3.120314,-0.003268,0.089513,-0.065836,0.0,0.712089,1.003198,0.159431,1.787587
143490047,0.295967,0.728097,0.296846,0.202522,0.464515,0.273338,0.576008,0.514773,0.596308,0.610194,...,2.545897,2.968346,0.213025,-0.089790,1.872033,0.0,1.035903,1.010666,0.018179,45.127029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114150551,0.392909,0.052322,0.297533,0.029816,0.349924,0.072254,0.026216,0.372755,0.030768,0.016076,...,2.349254,3.757998,0.145584,12.230811,-4.366978,0.0,0.761346,1.015743,0.545869,2.845130
620920171,0.025414,0.035201,0.030709,0.045806,0.047551,0.038767,0.008598,0.031370,0.021154,0.035638,...,2.659311,4.419274,-0.159470,3.974907,-9.215944,0.0,0.723719,1.006479,0.203513,2.437093
921110033,0.014278,0.018445,0.006360,0.023888,0.036698,0.101160,0.318794,0.723433,0.294589,0.385579,...,2.297598,3.391664,-0.087703,1.498308,1.349163,0.0,0.934436,1.018504,0.618581,2.466294
567868974,0.015523,0.190301,0.028636,0.217455,0.036683,0.101310,0.037358,0.104484,0.043239,0.039132,...,2.466197,5.015513,0.048049,3.303450,3.191335,0.0,0.676575,1.003214,0.127701,2.260325


In [11]:
df_all.to_csv('/media/jadhavlab/Data2/Volcanoes/test.csv')
